# Linear programming problem solving with the Gurobi library.

Library installation

In [ ]:
! pip install gurobipy
! pip install pandas

Libraries import

In [ ]:
import gurobipy as gp
import pandas as pd

Model programming

In [ ]:
# Number of variables in the problem
while True:
    try:
        num_variables = int(input("Enter the number of variables"))
        break
    except ValueError:
        print("Enter a valid number.")

# Ask for non-negativity restriction
free_vars = input("¿Are there variables without non-negativity restriction? If yes, input the number of that variables splitted by commas. If not, input '-': ").strip()

if free_vars == '-':
    free_vars = []
else:
    free_vars = [int(var) for var in free_vars.split(',') if 1 <= int(var) <= num_variables]

model = gp.Model("Modelo1")

# Variables creation
variables = []
for i in range(1, num_variables + 1):
    if i in free_vars:
        variables.append(model.addVar(lb=-gp.GRB.INFINITY, name=f"x{i}")) #Free var
    else:
        variables.append(model.addVar(lb=0.0, name=f"x{i}")) #no-negativity

#Objective function definition
tipo = input("Enter 'max' or 'min' depending on whether the objective function is to maximize or minimize: ").lower()
while tipo != "max" and tipo != "min":
    tipo = input("Introduce 'max' o 'min' según sea la funcion objetivo a maximizar o minimizar: ").lower()

while True:
    objective_input = input("Enter the coefficients of the objective function separated by commas: ")
    objective_coeffs = objective_input.split(',')
    
    if len(objective_coeffs) == num_variables:
        try:
            objective_coeffs = [float(coeff) for coeff in objective_coeffs]
            break
        except ValueError:
            print("Enter valid numerical coefficients")
    else:
        print(f"Enter {num_variables} coefficients in the correct format (examples: '3,1,2' or '3,-1,-8').")

model.setObjective(gp.quicksum(objective_coeffs[i] * variables[i] for i in range(num_variables)), sense=gp.GRB.MAXIMIZE if tipo == "max" else gp.GRB.MINIMIZE)

# Restrictions definition
while True:
    try:
        num_restricciones = int(input("Enter the number of restrictions: "))
        break
    except ValueError:
        print("Enter a valid integer number.")

for i in range(num_restricciones):
    restriccion = input(f"Enter the restriction number {i + 1} with the format '3*x1 - x2 + 4*x3 <= 10': ")
    if '<=' in restriccion:
        coeffs, rhs = restriccion.split('<=')
        relation = '<='
    elif '>=' in restriccion:
        coeffs, rhs = restriccion.split('>=')
        relation = '>='
    else:
        coeffs, rhs = restriccion.split('=')
        relation = '='

    coeffs = coeffs.split('+')
    coeffs = [c.strip() for c in coeffs] 

    # Restriction evaluation and adding to the model
    if relation == '<=':
        model.addConstr(gp.quicksum(eval(c, {}, {'x' + str(j): variables[j - 1] for j in range(1, num_variables + 1)}) for c in coeffs) <= float(rhs), name=f"restriction{i + 1}")
    elif relation == '>=':
        model.addConstr(gp.quicksum(eval(c, {}, {'x' + str(j): variables[j - 1] for j in range(1, num_variables + 1)}) for c in coeffs) >= float(rhs), name=f"restriction{i + 1}")
    elif relation == '=':
        model.addConstr(gp.quicksum(eval(c, {}, {'x' + str(j): variables[j - 1] for j in range(1, num_variables + 1)}) for c in coeffs) == float(rhs), name=f"restriction{i + 1}")

print(' ')
# Model resolution
model.optimize()

print(' ')
# Text format
bold = "\033[1m"
reset = "\033[0m"

#  Solution
if model.status == gp.GRB.OPTIMAL:
    print(f"{bold}Optimal solution found:{reset}")
    for i in range(num_variables):
        print(f"{variables[i].varName} = {variables[i].x}")
    print("Optimal objective value: ", model.objVal)
else:
    print("No optimal solution found.")

#Sensitivity analysis
# Reduced costs and ranges    
reduced_costs = model.getAttr('RC', model.getVars())    
reduced_cost_low = model.getAttr('SAObjLow', model.getVars())
reduced_cost_up = model.getAttr('SAObjUp', model.getVars())

# Dual values and ranges
dual_values = model.getAttr('Pi', model.getConstrs())
dual_range_low = model.getAttr('SARHSLow', model.getConstrs())
dual_range_up = model.getAttr('SARHSUp', model.getConstrs())

print(' ')

# Table 1 data
data1 = {
    "Variable name": [var.VarName for var in model.getVars()],
    "Actual coeff.": objective_coeffs,
    "Reduced cost": reduced_costs,
    "Lower limit": reduced_cost_low,
    "Upper limit": reduced_cost_up
}

# Table 1
df1 = pd.DataFrame(data1).set_index("Variable name")

# Table 2 data
data2 = {
    "Restriction name": [constr.ConstrName for constr in model.getConstrs()],
    "Current RHS": rhs,
    "Dual price": dual_values,
    "Lower limit": dual_range_low,
    "Upper limit": dual_range_up
}

# Table 2
df2 = pd.DataFrame(data2).set_index("Restriction name")


print(f"{bold} Reduced costs and allowable ranges of objective function coeff.: {reset}")
print(df1)

print(f"\n{bold}Dual prices and allowable ranges for RHS:{reset}")
print(df2)

model.dispose()